In [ ]:
import json
from tqdm import tqdm
import os
from cerebras.cloud.sdk import Cerebras
import time

api_keys = ["api_key1", "api_key2"] # ключи с сайта https://cloud.cerebras.ai/

api_keys = [key for key in api_keys if key.strip()]

current_key_index = 0

def get_client():
    global current_key_index
    if current_key_index >= len(api_keys):
        raise Exception("Все API ключи исчерпаны")
    
    return Cerebras(api_key=api_keys[current_key_index])

def switch_to_next_key():
    global current_key_index
    current_key_index += 1
    if current_key_index >= len(api_keys):
        return False
    print(f"[INFO] Переключение на API ключ #{current_key_index + 1}")
    return True

client = get_client()

jsonl_path = "train.jsonl"
results_path = "results.jsonl"

existing_ids = set()
if os.path.exists(results_path):
    with open(results_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line.strip())
                if isinstance(data, dict) and "id" in data:
                    existing_ids.add(data["id"])
            except json.JSONDecodeError:
                continue 

samples = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            sample = json.loads(line.strip())
            samples.append(sample)
        except json.JSONDecodeError:
            continue


# Генерация промпта с рассуждением
def build_prompt(sample):
    inputs = sample["inputs"]
    return (
        f"Инструкция для эксперта-аналитика**\n"
        f"Тема: '{inputs['subject']}'.\n"
        f"Ваша задача — выполнить строгий логический анализ предоставленной задачи. Следуйте этому алгоритму:\n"
        f"1.  **Анализ Задачи: Кратко определите основной вопрос и какой логический или математический принцип нужно применить.\n"
        f"2.  Пошаговая Оценка Вариантов: Систематически рассмотрите КАЖДЫЙ вариант ответа (A, B, C, D). Для каждого варианта предоставьте четкое и лаконичное объяснение, почему он является верным или неверным в контексте задачи.\n"
        f"3.  Синтез и Вывод: На основе пошагового анализа, сделайте окончательный вывод и выберите единственно правильный ответ.\n\n"
        f"Формат вывода**\n"
        f"Ваш ответ ДОЛЖЕН СТРОГО соответствовать формату ниже, без лишних вступлений или заключений:\n"
        f"**Рассуждение:**\n"
        f"[Здесь ваш детальный анализ по шагам 1-3]\n"
        f"**Ответ: [Здесь ОДНА буква: A, B, C или D]\n\n"
        f"---"
        f"**Задача для анализа:**\n"
        f"{inputs['text']}\n\n"
        f"**Варианты ответа:**\n"
        f"A) {inputs['option_a']}\n"
        f"B) {inputs['option_b']}\n"
        f"C) {inputs['option_c']}\n"
        f"D) {inputs['option_d']}\n\n"
    )

def process_samples():
    global client, current_key_index
    
    unprocessed_samples = [sample for sample in samples if sample["meta"]["id"] not in existing_ids]
    
    if not unprocessed_samples:
        print("[INFO] Все образцы уже обработаны")
        return
    
    progress_bar = tqdm(unprocessed_samples, desc="Processing")
    
    for sample in progress_bar:
        sample_id = sample["meta"]["id"]
        sample_domain = sample["meta"]["domain"]
        
        prompt = build_prompt(sample)
        
        success = False
        
        while not success and current_key_index < len(api_keys):
            max_retries = 3
            
            for retry in range(max_retries):
                try:
                    response = client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt}],
                        model="qwen-3-235b-a22b",#Llama-3.3-70B
                        temperature=0.5
                    )
                    answer = response.choices[0].message.content.strip()
                    success = True
                    break
                    
                except Exception as e:
                    print(f"[!] Ошибка на ID {sample_id} (ключ #{current_key_index + 1}, попытка {retry + 1}/{max_retries}): {e}")
                    
                    if retry < max_retries - 1:
                        time.sleep(2)
                        continue
            
            if not success:
                if switch_to_next_key():
                    client = get_client()
                    print(f"[INFO] Продолжаем с новым ключом для ID {sample_id}")
                else:
                    print("[ERROR] Все API ключи исчерпаны, обработка остановлена")
                    return
        
        if success:
            result = {
                "id": sample_id,
                "prompt": prompt,
                "model_answer": answer,
                "domain": sample_domain,
                "ground_truth": sample["outputs"]
            }
            
            with open(results_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(result, ensure_ascii=False) + "\n")
            
            existing_ids.add(sample_id)
            progress_bar.set_postfix({"current_key": current_key_index + 1, "total_keys": len(api_keys)})
        else:
            print(f"[ERROR] Не удалось обработать ID {sample_id} - все ключи исчерпаны")
            break

if __name__ == "__main__":
    print(f"[INFO] Начинаем обработку с API ключом #{current_key_index + 1}")
    process_samples()
    print("[INFO] Обработка завершена")